### LIRE LES FICHIERS

In [12]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Fichier des clusters IRIS
gdf_clusters = gpd.read_file("iris_clusters_200.gpkg")  # contient cluster_id et geometry

# Fichier des médecins avec latitude/longitude
df_med = pd.read_csv("offre_medical.csv")  # colonnes : latitude, longitude, specialité, etc.

# Transformer en GeoDataFrame
gdf_med = gpd.GeoDataFrame(
    df_med,
    geometry=gpd.points_from_xy(df_med.longitude, df_med.latitude),
    crs="EPSG:4326"  # WGS84
)

In [13]:
gdf_med_in_cluster = gpd.sjoin(gdf_med, gdf_clusters[['cluster_id', 'geometry']], how="left", predicate="within")

In [14]:
med_count_by_cluster = gdf_med_in_cluster.groupby('cluster_id').size().reset_index(name='med_count')

### TABLE POUR LA DATA VIZ

In [15]:
gdf_med_viz = gdf_med_in_cluster.merge(med_count_by_cluster, on='cluster_id', how='left')

# Colonnes utiles pour DataViz :
gdf_med_viz = gdf_med_viz[[
    'Libellé savoir-faire', 
    'cluster_id', 
    'med_count', 
    'longitude', 
    'latitude'
]]

In [20]:
import geopandas as gpd
from shapely.geometry import Point

# Si ton DataFrame contient 'longitude' et 'latitude'
gdf_med_viz_geo = gpd.GeoDataFrame(
    gdf_med_viz,  # ton DataFrame original
    geometry=gpd.points_from_xy(gdf_med_viz.longitude, gdf_med_viz.latitude),
    crs="EPSG:4326"  # système de coordonnées WGS84
)
# Maintenant tu peux sauvegarder en GeoPackage
gdf_med_viz_geo.to_file("medecins_clusters_viz.gpkg", driver="GPKG")

# Et tu peux aussi sauvegarder un CSV pour DataViz classique
gdf_med_viz_geo[['longitude', 'latitude', 'cluster_id', 'Libellé savoir-faire', 'med_count']].to_csv(
    "medecins_clusters_viz.csv", index=False )

In [1]:
import geopandas as gpd

# Charger ton GeoPackage
clusters = gpd.read_file("FV_medecins_clusters_viz.gpkg")

# Simplifier chaque polygone
# 0.001 correspond à la tolérance en degrés (~100m), tu peux ajuster
clusters["geometry"] = clusters["geometry"].simplify(tolerance=0.001, preserve_topology=True)

# Sauvegarder un GeoPackage simplifié
clusters.to_file("FV_medecins_clusters_viz_simplifie.gpkg", driver="GPKG")